In [1]:
import pandas as pd
import numpy as np
import os
path_year = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/SSC/Financial/Year/'
path_quar = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/SSC/Financial/Quarter/'
path_lake = '/content/drive/MyDrive/Data Lake/'
all_com = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase4.xlsx')['Symbol'])

In [2]:
file_balance = pd.read_excel('/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /data phase1/XuliExcel/Feature_Standard_Library.xlsx', sheet_name= 'Balance Sheet')
file_income = pd.read_excel('/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /data phase1/XuliExcel/Feature_Standard_Library.xlsx', sheet_name= 'Income Statement')
df_feature = pd.concat([file_balance, file_income]).reset_index(drop = True).rename(columns = {'Ma_BCTC':'code'})
df_feature  = df_feature.astype(str)

In [22]:
for filename in os.listdir(path_quar):
    df_remove = pd.read_csv(f'{path_quar}{filename}')
    if len(df_remove.index) == 0 and len(df_remove.columns) ==1:
        os.remove(f'{path_quar}{filename}')

#F1 year


In [ ]:
df_path_name = pd.DataFrame({'path_name':os.listdir(path_year)})
df_path_name['Symbol'] = df_path_name['path_name'].apply(lambda x:x[:3])
df_path_name = df_path_name.sort_values(by=['Symbol']).reset_index(drop = True)
list_com =list(df_path_name['Symbol'].unique())
df_com = df_path_name[df_path_name['Symbol']=='DTK'].reset_index(drop = True)

In [ ]:
def change_name(filename):
    df_ssc = pd.read_csv(f'{path_year}{filename}')
    df_ssc = df_ssc.dropna(how = 'all', axis = 1).rename(columns = {'1':'code', '3':filename[4:-4], '4':str(int(filename[4:-4])-1)})
    list_field = list(df_ssc['0'])
    id_cut = 0
    for id in range(len(list_field)):
        if 'lưu chuyển' in list_field[id].lower():
            id_cut = id
            break
    df_ssc = df_ssc.iloc[:id_cut].reset_index(drop= True)
    df_ssc = pd.merge(df_feature, df_ssc, how = 'left', on = ['code'])[['Feature', filename[4:-4], str(int(filename[4:-4])-1)]]
    return df_ssc

In [ ]:
def drop_col__(df):
    list_year_drop = []
    for i in df.columns:
        if '__' in i:
            list_year_drop.append(i)
    df = df.drop(columns = list_year_drop)
    return df

In [ ]:
def merge_company(symbol):
    df = df_feature[['Feature']]
    df_symbol = df_path_name[df_path_name['Symbol'] == symbol].reset_index(drop = True)
    df_year = pd.DataFrame()
    for i in df_symbol.index:
        df_tam = change_name(df_symbol['path_name'].iloc[i])
        df = pd.merge(df, df_tam, how = 'left', on = ['Feature'], suffixes = ('', '__'))
    return drop_col__(df)
merge_company('DTK')

In [ ]:
for symbol in list_com:
    print(symbol)
    merge_company(symbol).to_csv(f'/content/drive/MyDrive/Data Lake/Ingestion/Day 0/SSC/F1/Year/{symbol}.csv', index = False)

#F1 Quarter


In [80]:
df_path_name = pd.DataFrame({'path_name':os.listdir(path_quar)})
df_path_name['Symbol'] = df_path_name['path_name'].apply(lambda x:x[:3])
df_path_name['Time'] = df_path_name['path_name'].apply(lambda x:x[11:-4]+x[9])
df_path_name = df_path_name.sort_values(by=['Symbol', 'Time']).reset_index(drop = True)
list_com =list(df_path_name['Symbol'].unique())

In [ ]:
list_quy = []
for year in range(2018, 2022):
    for quy in range(1, 5):
        list_quy.append(f'{quy}/{year}')

In [91]:
def change_name_file_quy(filename):
    df_ssc = pd.read_csv(f'{path_quar}{filename}')
    quy = filename[9:-4].replace(' ', '/')
    quytruoc = list_quy[list_quy.index(quy)-1]
    df_ssc = df_ssc.rename(columns = {'1':'code', '3':quy, '4':quytruoc})
    list_field = list(df_ssc['0'])
    id_cut = 0
    for id in range(len(list_field)):
        if 'lưu chuyển' in list_field[id].lower():
            id_cut = id
            break
    df_ssc = df_ssc.iloc[:id_cut].reset_index(drop= True)
    df_ssc = pd.merge(df_feature, df_ssc, how = 'left', on = ['code'])[['Feature', quy, quytruoc]]
    df_ssc[quytruoc][117:] = np.nan
    return df_ssc
# change_name_file_quy(filename)

In [92]:
def drop_col__(df):
    list_year_drop = []
    for i in df.columns:
        if '__' in i:
            list_year_drop.append(i)
    df = df.drop(columns = list_year_drop)
    df = df.drop_duplicates().reset_index(drop = True)
    return df

In [ ]:
def merge_company(symbol):
    df = df_feature[['Feature']]
    df_symbol = df_path_name[df_path_name['Symbol'] == symbol].reset_index(drop = True)
    df_year = pd.DataFrame()
    for i in df_symbol.index:
        # print(i)
        df_tam = change_name_file_quy(df_symbol['path_name'].iloc[i])
        df = pd.merge(df, df_tam, how = 'left', on = ['Feature'], suffixes = ('', '__'))
    return drop_col__(df)
merge_company('BII')

In [ ]:
for symbol in list_com:
    print(symbol)
    merge_company(symbol).to_csv(f'/content/drive/MyDrive/Data Lake/Ingestion/Day 0/SSC/F1/Quarter/{symbol}.csv', index = False) 